# Detecting Anomalies In Purchased Electricity (Scope 2) Data Through Isolated Forests

## Package Importing

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest

## Database Importing

In [5]:
import dotenv
import os
import psycopg2

dotenv.load_dotenv(dotenv_path='database.env')

# --- Configuration ---
DB_NAME = os.environ['DB_NAME']
DB_USER = os.environ['DB_USER']
DB_PASS = os.environ['DB_PASS']
HOST = os.environ['HOST']
PORT = os.environ['PORT']

# 2. Database Connection and Setup
conn = psycopg2.connect(
    dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=HOST, port=PORT
)
conn.autocommit = True
cur = conn.cursor()

cur.execute('SELECT "facilityId","co2e","fuelType","createdAt","ch4","n2o","ef","co2" FROM purchased_electricity_activity')

df = pd.DataFrame(cur.fetchall(), columns = ["facilityId","co2e","fuelType","createdAt","ch4","n2o","ef","co2"])
print(df)

                                 facilityId         co2e  \
0      68f4851d-4959-49b6-96a1-63d80c816ed3     0.023896   
1      bf2278d8-0d33-4e28-a55e-ad3c1537af7e     0.002868   
2      bf2278d8-0d33-4e28-a55e-ad3c1537af7e     0.000810   
3      2af3b5a4-c50c-4e19-bf57-966f7d5eb4d0     0.657275   
4      2af3b5a4-c50c-4e19-bf57-966f7d5eb4d0     0.646236   
...                                     ...          ...   
29800  829c1cb0-918f-4a92-bdf0-9072faf8cf5f     1.309540   
29801  b4a32a71-f70c-4655-bb4f-94c2c96c27b5     0.177120   
29802  5b779dd3-79ad-4ffe-b961-69a20328d4a2     1.018440   
29803  a527d218-da19-45ea-9e0e-8f191d2ffc8a     0.503480   
29804  c062e8df-c057-43b7-8b2a-6beed46cf636  4275.815688   

                    fuelType                         createdAt       ch4  \
0                       AKMS  2025-01-05 22:00:45.524000-08:00  0.000001   
1                       AKMS  2025-01-19 09:40:06.100000-08:00  0.000000   
2                 QUEENSLAND  2025-01-19 18:43:21.6

## Data Cleaning

We will first create hour, day of week, and month-based variables to account for factors like seasonality.

In [7]:
# Convert 'createdAt' to datetime objects
df['createdAt'] = pd.to_datetime(df['createdAt'], utc=True)

# --- 2. Feature Engineering ---

# Extract time-based features
df['hour'] = df['createdAt'].dt.hour
df['day_of_week'] = df['createdAt'].dt.dayofweek # Monday=0, Sunday=6
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
df['month'] = df['createdAt'].dt.month

# For cyclical seasonality, convert month to sine and cosine features
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

This section of code factors in a manual "event log" that can account for changes in production, such as a facility being inactive during a certain period of time, or producing higher rates of a product during a certain busy time, and factoring that into the emission readings accordingly.

In [9]:
# --- 6. Event Log Integration and Feature Creation ---

# 1. Simulate the Event Log (This is where the user/system input goes)
# Example: Facility X increases production by 50% from June 1st to July 31st
event_log = pd.read_csv("sample_event_log.csv")

# 2. Initialize the new feature to the baseline (1.0, meaning no change)
df['production_boost_factor'] = 1.0

# 3. Apply the boost factor for records that fall within event periods
for _, event in event_log.iterrows():
    # Check if a record is from the facility AND within the event's date range
    mask = (df['facilityId'] == event['facilityId']) & \
           (df['createdAt'] >= event['event_start']) & \
           (df['createdAt'] <= event['event_end'])
    
    # Apply the magnitude to the new feature column
    df.loc[mask, 'production_boost_factor'] = event['magnitude']

print(f"Applied production boost to {df[df['production_boost_factor'] > 1.0].shape[0]} records.")

Applied production boost to 102 records.


We keep the necessary features in our data and prepare the data for our model.

In [11]:
# Define the features to be used in the model
NUMERICAL_FEATURES = ['co2e', 'ch4', 'n2o', 'ef', 'co2', 'month_sin', 'month_cos', 'production_boost_factor']
TIME_FEATURES = ['hour', 'day_of_week', 'is_weekend']
CATEGORICAL_FEATURES = ['facilityId', 'fuelType']

FEATURES = NUMERICAL_FEATURES + TIME_FEATURES + CATEGORICAL_FEATURES
X = df[FEATURES]

# --- 3. Preprocessing Pipeline ---

# Create transformers for different feature types
numerical_transformer = StandardScaler() # Scaling is crucial for Isolation Forest

# One-Hot Encoding for categorical features
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, NUMERICAL_FEATURES + TIME_FEATURES),
        ('cat', categorical_transformer, CATEGORICAL_FEATURES)
    ],
    remainder='drop' # Drop other columns not specified
)


## Model Creation

This section is where we define and create our model. We train it to prioritize recent data (such as in the last six months) and fit our data to the model. `X_recent` 

In [16]:
# The Isolation Forest model is typically trained on all data since we don't have labeled anomalies.
# 'contamination' is the expected proportion of anomalies in the dataset (e.g., 1%).
# Setting a reasonable contamination value helps the model set its internal threshold.

# Define the model within a pipeline for clean preprocessing and fitting
anomaly_detector = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', IsolationForest(
        n_estimators=100,
        contamination=0.01, # Set based on domain knowledge (e.g., 1% of data is anomalous)
        random_state=42,
        n_jobs=-1 # Use all available cores
    ))
])

# --- 5. Training and Prediction ---

print("Training Isolation Forest model...")

# Assuming 'df' is your historical data and you want to use the last 6 months
six_months_ago = pd.to_datetime('today', utc=True) - pd.DateOffset(months=6)

# Filter the data for the rolling window
recent_data = df[df['createdAt'] >= six_months_ago]
X_recent = recent_data[FEATURES]

anomaly_detector.fit(X)

# Get the anomaly scores (lower score means more anomalous)
# Note: Isolation Forest outputs a score where higher is 'normal'.
# We often use .decision_function() for the raw score.
df['anomaly_score'] = anomaly_detector.decision_function(X)

# Predict the anomaly classification (-1 for anomaly, 1 for normal)
df['anomaly_label'] = anomaly_detector.predict(X)

print("Training complete.")

Training Isolation Forest model...
Training complete.


## Results

In [20]:
print("\n--- Results Summary ---")
print(f"Total Anomalies Detected (-1): {df['anomaly_label'].value_counts().get(-1, 0)}")
print(f"Total Normal Observations (1): {df['anomaly_label'].value_counts().get(1, 0)}")

# --- 6. Inspection of Anomalies ---

anomalies = df[df['anomaly_label'] == -1].sort_values(by='anomaly_score')

# Show the top 5 most anomalous observations (lowest scores)
print("\nTop 5 Most Anomalous Records:")
print(anomalies[['createdAt','fuelType', 'co2e', 'anomaly_score']])

# Example of how to filter based on a score threshold if you prefer a continuous approach
# score_threshold = df['anomaly_score'].quantile(0.01) # Set threshold at the 1st percentile
# high_risk = df[df['anomaly_score'] < score_threshold]


--- Results Summary ---
Total Anomalies Detected (-1): 294
Total Normal Observations (1): 29511

Top 5 Most Anomalous Records:
                             createdAt               fuelType         co2e  \
19832 2025-04-24 15:16:22.060000+00:00  ELECTRICITY_FROM_GRID  2094.721778   
19852 2025-04-24 16:12:35.313000+00:00  ELECTRICITY_FROM_GRID  2774.705488   
9917  2025-04-24 16:12:35.313000+00:00  ELECTRICITY_FROM_GRID  2774.705488   
9897  2025-04-24 15:16:22.060000+00:00  ELECTRICITY_FROM_GRID  2094.721778   
29767 2025-04-24 15:16:22.060000+00:00  ELECTRICITY_FROM_GRID  2094.721778   
...                                ...                    ...          ...   
9869  2025-04-24 13:35:22.956000+00:00  ELECTRICITY_FROM_GRID   206.498960   
19804 2025-04-24 13:35:22.956000+00:00  ELECTRICITY_FROM_GRID   206.498960   
9850  2025-04-24 12:44:53.009000+00:00  ELECTRICITY_FROM_GRID   200.084100   
19785 2025-04-24 12:44:53.009000+00:00  ELECTRICITY_FROM_GRID   200.084100   
29720 2025-04-

In [26]:
print(anomalies['fuelType'].unique())

print(anomalies[anomalies['fuelType'] == "ELECTRICITY_FROM_GRID"].count())

['ELECTRICITY_FROM_GRID' 'HIOA' 'ERCT' 'NYLI']
facilityId                 141
co2e                       141
fuelType                   141
createdAt                  141
ch4                        141
n2o                        141
ef                         141
co2                        141
hour                       141
day_of_week                141
is_weekend                 141
month                      141
month_sin                  141
month_cos                  141
production_boost_factor    141
anomaly_score              141
anomaly_label              141
dtype: int64
